# Structured types

[Pierre-Antoine Champin](http://champin.net/)

http://github.com/pchampin/rust-w3c-2022

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/2.0/fr/"><img alt="Contrat Creative Commons" style="border-width:0" src="http://i.creativecommons.org/l/by-nc-sa/2.0/fr/88x31.png" /></a>

## Table of content

* [`struct` types](#struct)
* [Destructuring assignments](#destructuring)
* [`enum` types](#enum)
* [Methods](#methods)
* [Standard `enum`s](#std-enums)
* [Modules and visibility](#modules)

Background question: is Rust an object oriented programming language?

# `struct` types <a class="anchor" id="struct"></a>

Somewhere in-between C's structs and C++'s classes.

## Declaring a `struct` type

In [2]:
struct Person {
    given_name: String,
    family_name: String,
    age: u8,
}

struct Color {
    r: u8,
    g: u8,
    b: u8,
}

## Initializing a `struct`

In [3]:
let black = Color { b: 0, r: 0, g: 0 };

let blue = Color { b: 255, ..black };

let (r, g) = (12, 34);
let mut mycolor = Color { r, g, b: r+g };

## Accessing the fields of a `struct`

In [4]:
// reading
let x = mycolor.b;

// modifying
mycolor.r = x+1;

## Tuple-`struct` type

In [5]:
struct Rgb(u8, u8, u8);

let green = Rgb(0, 255, 0);

// fields are accessed as with tuples (.0, .1, ...)
let x = green.1;

## Zero-sized `struct` type

struct Foo;

let f = Foo;

In [6]:
struct Foo;

let f = Foo;

"What the heck would I do that for?"

→ we will see that later...

# Destructuring assignment (a.k.a. pattern matching) 🦀 <a class="anchor" id="destructuring"></a>

Instead of writing

Instead of writing

In [7]:
let r = mycolor.r;
let g = mycolor.g;
let b = mycolor.b;

we can write

In [8]:
let Color { r, g, b } = mycolor;

Instead of writing

In [9]:
let r1 = mycolor.r;
let g1 = mycolor.g;
let b1 = mycolor.b;

we can write

In [10]:
let Color { r: r1, g: g1, b: b1 } = mycolor;

Instead of writing

In [11]:
let r2 = green.0;
let g2 = green.1;
let b2 = green.2;

we can write

In [12]:
let Rgb(r2, g2, b2) = green;

...and by the way, this also works with tuples and arrays

In [13]:
let (a, b, c) = (1, 2, 3);
let [a, b, c] = [1, 2, 3];

## Partial destructuring assignment

In [14]:
let Color { b, g, .. } = mycolor;
println!("{} {}", b, g);

let Rgb(_, g, _) = green;
println!("{}", g);

let [first, _, third, .., last] = [1, 2, 3, 4, 5, 6, 7, 8, 9];
println!("{} {} {}", first, third, last);

46 34
255
1 3 9


## Recursive destructuring assignment

In [15]:
struct Gradient(Color, Color);

let g = Gradient(black, blue);

let Gradient(Color { r: r1, .. }, Color { b: b2, .. }) = g;
println!("{} {}", r1, b2);

0 255


## Destructuring assignment and move semantics

As any assignment in Rust, \
destructuring assignments have the move semantics \
(unless the type implements `Copy`).



In [16]:
let Gradient(c1, c2) = g;

NB: in the examples above (`Color`, `Rgb`), `u8` fields implement `Copy`.

## Destructuring references

In [17]:
{
    let mut g2 = Gradient(c1, c2);
    let Gradient(c3, c4) = &g2;
    // c3 and c4 are of type &Color
    println!("{} {}", c3.r, g2.0.r);
    
    let Gradient(c5, c6) = &mut g2;
    // c5 and c6 are of type &mut Color
    c5.b = 128;
    g2.0.b
}

0 0


128

# `enum` types 🦀 <a class="anchor" id="enum"></a>

A mix of C's `enum`s and `union`s.

## Declaring a simple `enum` type

In [18]:
enum Beatle {
    George,
    John,
    Paul,
    Ringo,
}

`George`, `John`... are called the **variants** of the `Beatle` type.

## Initializing a simple `enum`

In [19]:
let b1 = Beatle::John;

In [20]:
use Beatle::*;
let b2 = Paul;

## Using a simple `enum`

In [21]:
fn instrument(b: Beatle) -> &'static str {
    match b {
        George | John => "guitar",
        Paul => "bass",
        Ringo => "drums",
    }
}

## Declaring a complex `enum` type

In [22]:
struct Point { x: f64, y: f64 }

enum Figure2D {
    Plane,
    Line(Point, Point),
    Circle { centre: Point, radius: f64 },
}

## Initializing an `enum`

In [23]:
let p = Figure2D::Plane;
let l = Figure2D::Line(Point { x: 1.0, y: 2.0 }, Point { x: 3.0, y: 4.0 });

let centre = Point { x: 5.0, y: 6.0 };
let c = Figure2D::Circle{ centre, radius: 7.0 };

// p, l and c all have the same type: Figure2D
let figures = vec![p, l, c];

## Using an `enum` with `match`

In [24]:
use std::f64::consts::PI;

fn area(f: Figure2D) -> f64 {
    use Figure2D::*;
    match f {
        Plane => f64::INFINITY,
        Line(..) => 0.0,
        Circle { radius: r, .. } => PI*r*r,
    }
}

## `if let` statement

In [25]:
use Figure2D::*;

if let Line(p1, p2) = &figures[0] {
    // do something with p1 and p2
}

// more concise that:
match &figures[0] {
    Line(p1, p2) => { /* do something with p1 and p2 */ }
    _ => {}
}

()

NB: there also exists a `while let` statement.

# Methods  <a class="anchor" id="methods"></a>

## All types have methods in Rust 🦀

... even atomic types.

E.g

In [26]:
let x = "hello".len();
let y = 42.max(0);
let z = 3.14_f64.log2();

## Defining methods for a user-defined type

In [27]:
impl Color {
    /// Non-mutating method, borrowing self
    fn saturation(&self) -> f64 {
        let cmax = self.r.max(self.g).max(self.b);
        if cmax == 0 {
            0.0
        } else {
            let cmax = cmax as f64; // shadowing
            let cmin = self.r.min(self.g).min(self.b) as f64;
            (cmax - cmin) / cmax
        }
    }
    
    /// Mutating method, borrowing self mutably
    fn lighten(&mut self, delta: u8) {
        self.r = self.r.saturating_add(delta);
        self.g = self.g.saturating_add(delta);
        self.b = self.b.saturating_add(delta);
    }
}

In [28]:
let mut c = Color { r: 180, g: 0, b: 0};
println!("{}", c.saturation());
c.lighten(127);
println!("{}", c.saturation());

1
0.5019607843137255


## More methods and associated functions

In [29]:
impl Color { // there can be more than one 'impl' blocks for a type

    /// "Consuming" method, moving self
    fn to_rgb(self) -> Rgb {
        let Color { r, g, b } = self;
        Rgb(r, g, b)
    }
    
    /// Associated function, without a 'self' parameter.
    /// Common pattern for "constructor"
    fn new(r: u8, g: u8, b: u8) -> Color {
        Color { r, g, b }
    }
}

In [30]:
let navy = Color::new(0, 0, 128);
let x: Rgb = navy.to_rgb(); 
//println!("{}", navy.saturation()); // does not compile
println!("{} {} {}", x.0, x.1, x.2);

0 0 128


# Standard `enums` 🦀  <a class="anchor" id="std-enums"></a>

## The `Option<T>` type 🦀

### Motivation

* In Java, a function with return type `MyObject` may return an instance of `MyObject` *or* `null`. (Same in C with a function returning a pointer)

* This is a breach in the type system, because `null` is *not* of type `MyObject`:

    + it does not have the method of that class,
    + and any attempt to use it as a `MyObject` will raise an error (`NullPointerException`)

* In Rust, a function with return type `T` **must** return a `T` value.

* There are however cases where one might want to return "a `T` or nothing". Examples:

  + the `pop()` method of `Vec<T>` removes the last element of the vector and returns it, but does nothing if the vector is empty.
  
  + the `get(i)` method of `[T]` returns the element at index *i* if it exists, or nothing if *i* is too big.

* The return type of those methods is `Option<T>`, an `enum` with two variants:

  + `None`, representing "no value"
  + `Some(t)`, where *t* is a `T` value

### `Option<T>` in action

In [31]:
/// Retun the sum of all positive values in v
fn foo(mut v: Vec<i32>) -> i32 {
    let mut s = 0;
    loop {
        match v.pop() {
            None => {
                return s;
            }
            Some(i) if i > 0 => {
                s += i;
            }
            _ => {}
        }
    }
}

foo(vec![2, -3, 4])

6

Other implémentation, using `while let`

In [32]:
/// Retun the sum of all positive values in v
fn foo(mut v: Vec<i32>) -> i32 {
    let mut s = 0;
    while let Some(i) = v.pop() {
        if i > 0 {
            s += i;
        }
    }
    return s;
}

foo(vec![2, -3, 4])

6

### The `unwrap` method

When we are confident that the option is not `None`,
we can use the `unwrap` method:

* for `Some(t)`, it returns `t`;
* for `None`, it stops the program (panic) with an error message.

In [33]:
let mut v = vec![1,1,2,3,5];
let val: i32 = v.pop().unwrap();
val

5

## The `Result<T,E>` type 🦀

### Motivation

* Some functions may succeed (and return their result normally) or *fail*.

  + In the latter case, we might want to know the reason of the failure.
  
* In Rust, these functions return a `Result<T,E>`, an `enum` with the following variants:

  + `Ok(t)` where *t* is a `T` value, representing the normal outcome of the function;
  + `Err(e)` where *e* is an `E` value, representing an abnormal outcome.

### `Result<T, E>` in action

In [34]:
use std::fs::read

match read("README.md") {
    Ok(content) => println!("Content: {} bytes", content.len()),
    Err(e) => println!("ERROR: {}", e),
}

Content: 681 bytes


()

NB: the `Result<T,E>` also has an `unwrap` method,\
unwrapping the `Ok` value and panicking for `Err`.

In [35]:
let content = read("README.md").unwrap();
println!("{} bytes", content.len());

681 bytes


### Relaying errors

In [36]:
use std::fs::{read, write};

/// Copy at most 42 bytes from a file into another one,
/// and return the number of characters copied
fn copy_start(filename1: &str, filename2: &str) -> Result<usize, std::io::Error> {
    let mut content = match read(filename1) {
        Ok(c) => c,
        Err(e) => return Err(e),
    };
    content.truncate(42);
    match write(filename2, &content) {
        Ok(_) => {},
        Err(e) => return Err(e),
    }
    Ok(content.len())
}

* pros: `Result`s force you to handle errors
* cons: error handling in interleaved with the normal behaviour,\
  hurting readability

### Relaying errors the Rust way 🦀

In [37]:
use std::fs::{read, write};

/// Copy at most 42 characters from a file into another one,
/// and return the number of characters copied
fn copy_start(filename1: &str, filename2: &str) -> Result<usize, std::io::Error> {
    let mut content = read(filename1)?;
    content.truncate(42);
    write(filename2, &content)?;
    Ok(content.len())
}

# Modules and visibility <a class="anchor" id="modules"></a>


## Hierarchichal structure of a program

In [38]:
fn main() {
    /*...*/
}

mod foo {
    fn f1() { /*...*/ }
    
    mod bar {
        fn f2() { /*...*/ }
    }
}

mod baz {
    fn f3() { /*...*/ }
}

```rust
////// main.rs               ////// foo.rs
fn main() { /*..*/ }         fn f1() { /*...*/ }
mod foo;                    mod bar;
mod baz;

//// baz.rs                 ///// foo/bar.rs
fn f3() { /*...*/ }          fn f2() { /*...*/ }
```

## Visibility

The following elements :

* functions and methods,
* `struct` and `enum` types,
* fields of `struct`s,
* modules,

are only visible in the module defining them (and its submodules) by default.

Adding the `pub` (public) keyword before their declaration\
makes them visible from outside.

In [39]:
pub mod foo {
    pub fn fn1(x: f64) -> f64 { x*fn2(x) }
    fn fn2(x: f64) -> f64 { if x<0.0 { -x } else { x } }
    
    pub struct Foo {
        pub f1: f64,
        f2: f64,
    }
    impl Foo {
        pub fn new(f1: f64, f2: f64) -> Foo { Foo{ f1, f2, } }
        pub fn f2(&self) -> f64 { self.f2 }
        pub fn prod(&self) -> f64 { self.f1*self.f2 }
    }
}

In [40]:
let x = foo::Foo::new(-2.0, 1.0);
foo::fn1(x.f2 + x.prod())

Error: field `f2` of struct `foo::Foo` is private